<a href="https://colab.research.google.com/github/rootdrew27/cyberbullying-ml/blob/main/light_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=4ea980aa0802acc004afc95da4781e7ef7650332412ff31bc41de4606bc8a48b
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [14]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from langdetect import detect, LangDetectException

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataPath = '/content/drive/MyDrive/UWEC/ML Research w Dr. Vanamala/Data/cyberbullying_tweets.csv'

In [5]:
df = pd.read_csv(dataPath)

In [7]:
df.iloc[29205][0] # uh oh...

'is feminazi an actual word with a denot…\r\n@Nasharchy my job means protecting people. even the people i don\'t agree with/like.\r\nThey barely cooked anything on that entrée #MKR\r\nRT @kf: Like, no. If a community fostering jerks prevents users from adopting that community\'s code, it is very much a technical problem.\r\nIt\'s almost time. :)\r\n@JamesGweenwood shock youre bullying me again.\r\n@LionLionEatEat yeah, someone DMed me a screenshot. meh. let her. i have no idea who she even is.\r\nFor more egregious cases for harassment, we definitely need to be able to respond to those appropriately. But most toxicity isn\'t this.\r\nSo, this happened. http://t.co/ZPTRtsYFiV\r\n@Nibelsnarfabarf @srhbutts @GRIMACHU it is really funny all the assumptions they make about how it works - and how much they are wrong.\r\nThe lack of self-awareness from @wadhwa right now is staggering and hilarious. #stopwadhwa2015\r\nHahahaha http://t.co/JRPkJCNVH9\r\nThis is going to be a better finale than 

In [8]:
# tweets with \r\n were not properly read (or written) thus we will remove all tweets with '\r\n' as they are likely
df1 = df.copy()
df1 = df1[df1.tweet_text.str.contains('\r\n') == False]

In [ ]:
# note that tweets can technically contain more than 280 characters as links will always count as 22 characters
# source: https://smallbusiness.chron.com/doesnt-count-characters-twitter-77095.html

In [18]:
# cleaning functions

# remove links, mentions, RT flags, and punctuation.

def standardize_text(text):
  return text.lower()

def remove_non_english(text):
    try:
        lang = detect(text)
    except LangDetectException:
        lang = "unknown"
    return text if lang == "en" else ""

def remove_links(text):
    url_pattern = re.compile(r'(?:http[s]?://)?(?:www\.)?(?:bit\.ly|goo\.gl|t\.co|tinyurl\.com|tr\.im|is\.gd|cli\.gs|u\.nu|url\.ie|tiny\.cc|alturl\.com|ow\.ly|bit\.do|adoro\.to)\S+') #remove url shorteners
    return url_pattern.sub('', text)

def remove_mentions(text):
    return re.sub(r'@\w+', '', text)

def remove_rt(text):
    return re.sub(r'rt', '', text)

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

def remove_short_tweets(text):
    return text if len(text.split()) > 2 else ""

def remove_excess_spaces(text):
    return re.sub(r'\s+', ' ', text)


In [19]:
def preprocess(text):
  text = standardize_text(text)
  text = remove_links(text)
  text = remove_mentions(text)
  text = remove_rt(text)
  text = remove_non_english(text)
  text = remove_punctuation(text)
  text = remove_short_tweets(text)
  text = remove_excess_spaces(text)
  return text

In [20]:
df1.tweet_text = df1.tweet_text.apply(preprocess)

,tweet_text,cyberbullying_type
0,in other words katandandre your food was crapi...,not_cyberbullying
1,why is aussietv so white mkr theblock imaceleb...,not_cyberbullying
2,a classy whore or more red velvet cupcakes,not_cyberbullying
3,meh p thanks for the heads up but not too con...,not_cyberbullying
4,this is an isis account pretending to be a ku...,not_cyberbullying


In [24]:
df1.dropna(axis=0, inplace=True) #drop rows that contain any null values
df1.reset_index(drop=True) #reset the indexes

,tweet_text,cyberbullying_type
0,in other words katandandre your food was crapi...,not_cyberbullying
1,why is aussietv so white mkr theblock imaceleb...,not_cyberbullying
2,a classy whore or more red velvet cupcakes,not_cyberbullying
3,meh p thanks for the heads up but not too con...,not_cyberbullying
4,this is an isis account pretending to be a ku...,not_cyberbullying
...,...,...
47664,black ppl arent expected to do anything depend...,ethnicity
47665,turner did not withhold his disappointment tur...,ethnicity
47666,i swear to god this dumb nigger bitch i have g...,ethnicity
47667,yea fuck you if youre a nigger fucking unfollo...,ethnicity


In [25]:
df1.to_csv('clean_data_light.csv', index=False)